# Coordinate transformations

## Motivation

In all fields of science we frequently encounter data that is represented in coordinates or coordinate systems that are not apt for certain operations or visualizations.
In these cases we may thus need to compute new coordinates based on one or multiple existing coordinates.
For simple cases this may just be done by hand.
Consider:

In [ ]:
import scipp as sc
import plopp as pp

x = sc.linspace(dim='x', unit='m', start=1.0, stop=55.0, num=100)
da = sc.DataArray(data=x * x, coords={'x': x})
da.plot(figsize=(4, 3))

We may want to use $x^2$ instead of $x$ as a coordinate, to highlight the quadratic nature of our data:

In [ ]:
da2 = da.copy()
da2.coords['x_square'] = x * x
da2

While adding a new coordinate may often be done with a single line of code, the above example highlights the first shortcoming of this approach:
To actually visualize `da` using this new coordinate we must additionally rename the dimension:

In [ ]:
da2 = da2.rename_dims({'x': 'x_square'})
da2.plot(figsize=(4, 3))

Further complications are:

- The original coordinate is preserved and may get in the way in subsequent operations.
- Event-coordinates of binned data are not handled.
- Multi-step conversions with multiple inputs and multiple outputs may be required in practice.

To accommodate these recurring yet highly application-specific needs, Scipp provides a generic mechanism for transforming coordinates.
This is described and exemplified in the following.

## `transform_coords`

### Overview

[sc.transform_coords](../generated/functions/scipp.transform_coords.rst#scipp.transform_coords) (also available as method of data arrays and datasets) is a tool for transforming one or more input coordinates into one or more output coordinates.
It automatically handles:

- Renaming of dimensions, if dimension-coordinates are transformed.
- Making coordinates unaligned to avoid interference of coordinates consumed by the transformation in follow-up operations.
- Conversion of event-coordinates of binned data, if present.

### Basic example

We start by revisiting the example given in [Motivation](#Motivation).
The building blocks `transform_coords` operates on are functions with named parameters.
The parameter names define the names of the *input coordinates to consume*.
Let us define `x_square`, which will *consume* `x`:

In [ ]:
def x_square(x):
    return x * x

Next, we create a `dict`, mapping from an output coord name to a function that can create this coordinate.
The [sc.show_graph](../generated/functions/scipp.show_graph.rst#scipp.show_graph) helper is a convenient tool for visualizing the coordinate transformation defined by such as mapping:

In [ ]:
graph = {'x^2': x_square}
sc.show_graph(graph)

Here, the `x` coordinate can be consumed by the `x_square` function, creating the `x^2` coordinate.
Note that the function name and coordinate are unrelated.
Next, we can call `transform_coords`.
Apart from the graph, we also pass a list of desired output coordinates, here simply `['x^2']`.
`transform_coords` returns a new (shallow-copied) data array with added coordinates:

In [ ]:
transformed = da.transform_coords(['x^2'], graph=graph)
transformed

Note how `x` is now unaligned (not shown in bold), i.e., operations will not use it for alignment anymore.
This is important since it will allow for operations combining `transformed` with other data that may have matching `x^2` but not `x`.

## Example: Multi-step transform splitting and combining input coords

### Introduction

Let us consider a more complex example.
Imagine we have sensors around the globe, counting lightning strikes.
For each sensor get have data recorded at a certain UTC, and the sensor location.
We may be interested in variation of lightning strike frequency with time of day, as well as latitude.
To obtain this, we must:

1. Extract latitude and longitude information from the sensor locations.
2. Compute the local datetime from the datetime and a "timezone" offset from the longitude.
3. Extract the time from the local datetime.

For this purpose, we may define functions that look as follows.
We suggest ignoring the implementation details of these functions, since they are approximations and irrelevant for this example:

In [ ]:
def lat_long(location):
    x = location.fields.x
    y = location.fields.y
    z = location.fields.z
    theta = sc.to_unit(sc.atan2(y=sc.sqrt(x * x + y * y), x=z), 'deg', copy=False)
    phi = sc.to_unit(sc.atan2(y=y, x=x), 'deg', copy=False)
    return {'latitude': 90.0 * sc.Unit('deg') - theta, 'longitude': phi}


def local_datetime(datetime, longitude):
    long = sc.to_unit(longitude, unit='deg', copy=False)
    angular_velocity = (360.0 * sc.Unit('deg')) / (24.0 * sc.Unit('hour'))
    offset = (long / angular_velocity).astype('int64') + 12 * sc.Unit('hour')
    return sc.to_unit(offset, datetime.unit) + datetime


def time(local_datetime):
    seconds_per_day = sc.scalar(24 * 60 * 60, unit='s')
    start_day = sc.scalar(start.value.astype('datetime64[D]'))
    start_day_in_seconds = sc.scalar(start_day.values.astype('datetime64[s]'))
    offset = local_datetime - start_day_in_seconds
    time = (offset % seconds_per_day).astype('float64')
    return time

### Defining a transformation graph

Based on these functions we may then create a mapping between coordinate names and functions.
The visualization of the graph gives a handy summary of the desired conversion outlined above:

In [ ]:
graph = {
    (
        'longitude',
        'latitude',
    ): lat_long,
    'local_time': time,
    'local_datetime': local_datetime,
}
sc.show_graph(graph, size='6')

### Sample data

Next, let us look at the data we are working with.
Here we simply create some fake data, the details of the following code cell are irrelevant and should also be ignored:

In [ ]:
import numpy as np

hour_steps = sc.arange(
    dim='datetime',
    dtype='int64',
    unit='s',
    start=0,
    stop=3 * 24 * 60 * 60,
    step=60 * 60,
)
start = sc.scalar(np.datetime64('2021-06-01T17:00:00'))
datetime = start + hour_steps
nsite = 1000
ntime = len(datetime)
# Note that these points are NOT uniformly distributed on a sphere, this is NOT a good way to generate such points
location = sc.vectors(dims=['location'], values=np.random.rand(nsite, 3)) - sc.vector(
    value=[0.5, 0.5, 0.5]
)
location *= 6371 * sc.Unit('km') / sc.norm(location)
da = sc.DataArray(
    data=sc.array(dims=['location', 'datetime'], values=np.random.rand(nsite, ntime)),
    coords={'location': location, 'datetime': datetime},
)
north = location.fields.z > 0.0 * sc.Unit('km')
north.unit = sc.units.one
da += 2.0 * (north).astype('float64')  # more lightning strikes in northern hemisphere
phi0 = sc.atan2(y=location.fields.y, x=location.fields.x) - sc.to_unit(
    90.0 * sc.Unit('deg'), 'rad'
)
sin = sc.sin(
    phi0 + sc.linspace(dim='datetime', unit='rad', start=0, stop=6 * np.pi, num=ntime)
)
da += 2 * (sin + 1)  # more lightning strikes later in the day
da.unit = 'counts'

Our input data looks as follows, a 2-D data array with dimensions `datetime` and `location`, and corresponding coordinates:

In [ ]:
da

A 3-D scatter plot may be used to visualize this.
When dragging the `datetime` slider we can observe how the lightning counts shifts around the globe with the time of the day (the fake data covers a period of 3 days).
Note that the slider is only functional when running the notebook and is not functional in the online documentation page:

In [ ]:
def scatter_plot(da):
    from plopp import widgets as pw

    da = da.copy(deep=False)
    da.coords['x'] = da.coords['location'].fields.x
    da.coords['y'] = da.coords['location'].fields.y
    da.coords['z'] = da.coords['location'].fields.z
    inp = pp.Node(da)
    slider = pw.SliceWidget(da, dims=['datetime'])
    slider_node = pp.widget_node(slider)
    slice_node = pw.slice_dims(inp, slider_node)
    fig = pp.scatter3dfigure(slice_node, x='x', y='y', z='z', pixel_size=500, cbar=True)
    return pw.Box([fig, slider])


scatter_plot(da)

### Performing a transformation

With this setup, the actual coordinate transformation is now very simple:

In [ ]:
transformed = da.transform_coords(['latitude', 'local_time'], graph=graph)

The result is:

In [ ]:
transformed

In the above:

- `latitude` and `local_time` coordinates have been computed as requested.
- The intermediate results `local_datetime` and `longitude` were preserved as unaligned coordinates (use `keep_intermediate=False` to drop them).
- The `location` and `datetime` coordinates (which have been consumed by the transformation) have been converted to unaligned coordinates (use `keep_inputs=False` to drop them).
- The `datetime` *dimension* has been consumed by the `local_time` coordinate and thus renamed to `local_time` (use `rename_dims=False` to disable).
  For more details see section [Renaming of Dimensions](#Renaming-of-Dimensions).

### Post-processing

In some cases the above result may be all we need.
Frequently however, we may need to resample or bin our data after this coordinate transformation.

In the above case, `local_time` is now a 2-D coordinate, and the coordinate is not ordered since the "date" component of the datetime has been removed.
We may thus want to bin this data into latitude/local_time bins.
Here we first use `flatten` with a dummy dimension to make the data suitable for `sc.bin`:

In [ ]:
time_edges = sc.linspace(dim='local_time', unit='s', start=0, stop=24 * 60 * 60, num=6)
latitude = sc.linspace(dim='latitude', unit='deg', start=-90, stop=90, num=13)
binned = sc.bin(transformed.flatten(to='dummy'), latitude=latitude, local_time=time_edges)

The result looks as follows.
If this was real data (the sample data is fake!) we might observe that there are more lightning strikes on the northern hemisphere as well as later in the day.
This might be attributed to more thunderstorms after hot summer days.
Note that this example does not represent reality and is merely meant to illustrate several concepts of `transform_coords`:

In [ ]:
binned.hist(latitude=36, local_time=24).plot()

## Renaming of Dimensions

This section is somewhat advanced and not required to understand the basic usage of `transform_coords.`

As shown above, `transform_coords` can rename dimensions of the data array if it processes dimension-coordinates.
The rules controlling it are described in this section.
They are generic and tend to favor not renaming a dimension if it is not entirely clear what the new name should be.

A dimension is only renamed if its dimension-coordinate can be uniquely associated with one output coordinate of `transform_coords`.
In the example above, there are two dimension-coordinates in the input, `datetime` and `location`.
The latter is used to compute two new coordinates, namely `longitude` and `latitude`.
It is thus not possible to find a unique new name for the `location` *dimension*.
`datetime` on the other hand is used to construct only a single coordinate, `local_datetime`, which is finally transformed into `local_time`.
The `datetime` *dimension* is therefore renamed to `local_time`.

Identifying outputs with dimension-coordinates can be expressed as a graph-coloring problem.
We assign a unique color to each dimension-coordinate (coordinate name matching a dimension of the coordinate); other coordinates are left uncolored (black).
We then propagate colors through the directed graph using the following rules

1. If a node has exactly one colored parent, use that parent's color (graph 1).
2. If a node has several colored parents, leave the node black (graph 2).
   The same happens if no parent is colored.
3. If a node has more than one child, it is not counted for rules 1 and 2; its color is not applied to its children (graph 3).
   Other colored nodes are free to apply their color to shared children (graph 4).
   It makes no difference if the children are computed by the same function or from multiple functions.

![base rules](../images/transform_coords/base_rules.svg)

All graphs used by `transform_coords` are directed acyclic graphs, but they can still have undirected cycles.
In those cases, it can be possible to identify an output with a dimension-coordinate even when it has multiple children as part of a cycle.

4. If a cycle has one and only one colored node as input (w.r.t. directions of the edges) and exactly one output, that output takes the color of the input.
   In example 1 below, `a` can be uniquely associated with `c`.
   But example 2, this is not possible because there are two final nodes that depend on `a`.

![cycle rules](../images/transform_coords/cycle_rules.svg)

The following graphs illustrate how the rules interact in larger examples.
In particular, it shows that dimensions are renamed to the 'farthest away' coordinate even if that is not a final result of the transformation.

- In graph 1, `a` is renamed to `c` because of rule 1.
  `d` is renamed to `f` because of the second part of rule 3.
  `g` cannot be renamed because rule 2 applies to its only child, `h`.
- In graph 2, `a` is renamed to `h` because `c` and `h` can be associated with each other through the cycle according to rule 4.
- This is no longer the case when `d` is also a dimension-coordinate (graph 3).
  Like in example 1, `a` is renamed to `c` and `d` to `f`.
  But `h` depends on `f` directly and `c` through the cycle and can therefore not be associated with a single dimension.

![larger examples](../images/transform_coords/larger_examples.svg)

More details on how the algorithm works and the rationale behind it can be found in [Architecture Decision Record 0011](../development/adr/0011-renaming-of-dimensions-in-transform_coords.rst).